In [ ]:
import pandas as pd
from rag_eval import RAGEvaluator, RAGEvalConfig, load_pipeline, load_eval_dataframe  
from langchain_community.llms.sambanova import SambaStudio
from langchain_community.embeddings import HuggingFaceInstructEmbeddings

In [ ]:
# Load config
config = RAGEvalConfig("config.yaml")

In [ ]:
# Create evaluator
eval_llms = [SambaStudio(**conf) for conf in config.eval_llm_configs]
eval_embeddings = HuggingFaceInstructEmbeddings(model_name=config.embedding_model_name)
evaluator = RAGEvaluator(eval_llms, eval_embeddings, "config.yaml")

In [ ]:
# Use Case 1: CSV file with pre-generated answers, no context
eval_df = pd.read_csv("eval_set_with_answers.csv")
results1 = evaluator.evaluate(eval_df)
print("Results 1:", results1)


In [ ]:
# Use Case 2: CSV file with pre-generated answers and context  
eval_df = pd.read_csv("eval_set_with_answers_and_context.csv")
results2 = evaluator.evaluate(eval_df)
print("Results 2:", results2)

In [ ]:
# Use Case 3: CSV file without answers, generate with pipelines, no context
eval_df = pd.read_csv("eval_set.csv")
pipelines = [load_pipeline(SambaStudio(**conf), config) for conf in config.llm_configs]  
results3 = evaluator.evaluate(eval_df, pipelines)
print("Results 3:", results3)

In [ ]:
# Use Case 4: CSV file without answers, generate with pipelines, with context from vector DB
eval_df = pd.read_csv("eval_set.csv")
pipelines = [load_pipeline(SambaStudio(**conf), config) for conf in config.llm_configs]
results4 = evaluator.evaluate(eval_df, pipelines) 
print("Results 4:", results4)

In [ ]:
# Use Case 5: Evaluate on HF dataset
config.config["eval_dataset"]["hf_dataset_name"] = "squad_v2"
eval_df = load_eval_dataframe(config)
pipelines = [load_pipeline(SambaStudio(**conf), config) for conf in config.llm_configs] 
results5 = evaluator.evaluate(eval_df, pipelines)
print("Results 5:", results5)